## Задание 1 

*Максимальная оценка задания: **1** балл*

Дан набор данных:

| tid              | itemset |
| ---------------- | ------- |
| *t<sub>1</sub>*  | *CDE*    |
| *t<sub>2</sub>*  | *BC*    |
| *t<sub>3</sub>*  | *ADE*    |
| *t<sub>4</sub>*  | *ABCDE*    |
| *t<sub>5</sub>*  | *BD*    |
| *t<sub>6</sub>*  | *AE*    |
| *t<sub>7</sub>*  | *ACDE*    |
| *t<sub>8</sub>*  | *ACDE*    |

Минимальный уровень поддержки = **4**

1.1 Найдите частые наборы. Для поиска воспользуйтесь собственной реализацией **Apriori** из работы №3 (демонстрация работы алгоритма не требуется)

1.2 Найдите все минимальные генераторы. Оформите этот пункт в виде функции

1.3 К исходным данным добавьте набор ***E*** и повторно найдите все минимальные генераторы. Значение минимального уровня поддержки оставьте без изменений


In [1]:
import pandas as pd
import numpy as np

In [2]:
from collections import defaultdict

def printResults(itemset,tail=50):
    if itemset != {}:
        df=pd.DataFrame()
        df['itemset']=[list(item) for item in itemset.keys()]
        df['support']=itemset.values()
        df['length'] = df['itemset'].apply(lambda x: len(x))
        print(df.tail(tail))

def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet): 
        localSet = {}
        for item in itemSet:
                for transaction in transactionList:
                        if item.issubset(transaction):
                                freqSet[item] += 1 
                                localSet.setdefault(item, 0)
                                localSet[item] += 1
        count=10
        localSet = {item:val for item,val in localSet.items() if val/count >= minSupport}
        return localSet


def joinSet(itemSet, length):
        return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])

def getItemSetTransactionList(itemset):
    transactionList = list()
    itemSet = set()
    for record in itemset:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))  
    return itemSet, transactionList


def runApriori(dataset, minSupport):
    
    itemSet, transactionList = getItemSetTransactionList(dataset)
    freqSet = defaultdict(int)
    resultSet = {}
    currentLSet=returnItemsWithMinSupport(itemSet, transactionList, minSupport,freqSet)
    uniqSet=currentLSet
    resultSet.update(currentLSet)
    currentLSet = set(currentLSet.keys())
    k = 2
    while currentLSet != set([]):
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet, transactionList, minSupport, freqSet)
        resultSet.update(currentCSet)
        currentLSet = set(currentCSet)
        k += 1
    return resultSet,uniqSet


itemset=['CDE','BC','ADE','ABCDE','BD','AE','ACDE','ACDE']
minSupport = 4/10
print('Data:', itemset)
print('Minsup ratio:', minSupport)
    
result,uniqSet = runApriori(itemset, minSupport)
print('\n Apriori result:')
printResults(result)



Data: ['CDE', 'BC', 'ADE', 'ABCDE', 'BD', 'AE', 'ACDE', 'ACDE']
Minsup ratio: 0.4

 Apriori result:
      itemset  support  length
0         [E]        6       1
1         [D]        6       1
2         [A]        5       1
3         [C]        5       1
4      [D, C]        4       2
5      [D, E]        5       2
6      [E, C]        4       2
7      [D, A]        4       2
8      [A, E]        5       2
9   [D, E, C]        4       3
10  [D, A, E]        4       3


In [3]:
def min_gen(itemset,generset):
    for item,support in itemset.items():
        check=1
        for g_item,g_support in generset.items():
            if g_item.issubset(item):
                if support>=g_support:
                    check=0
        if check==1: 
            generset.setdefault(item,support)
    return generset
print('\n Min gen result:')
printResults(min_gen(result,uniqSet))
    


 Min gen result:
  itemset  support  length
0     [E]        6       1
1     [D]        6       1
2     [A]        5       1
3     [C]        5       1
4  [D, C]        4       2
5  [D, E]        5       2
6  [E, C]        4       2
7  [D, A]        4       2


In [4]:
print('\n Add E:')
itemset.append('E')
print(itemset)
newResult,newUniqSet=runApriori(itemset,0.4)
print('Apriori result:')
printResults(newResult)
print('\n Min gen result:')
printResults(min_gen(newResult,newUniqSet))


 Add E:
['CDE', 'BC', 'ADE', 'ABCDE', 'BD', 'AE', 'ACDE', 'ACDE', 'E']
Apriori result:
      itemset  support  length
0         [E]        7       1
1         [D]        6       1
2         [A]        5       1
3         [C]        5       1
4      [D, C]        4       2
5      [D, E]        5       2
6      [E, C]        4       2
7      [D, A]        4       2
8      [A, E]        5       2
9   [D, E, C]        4       3
10  [D, A, E]        4       3

 Min gen result:
  itemset  support  length
0     [E]        7       1
1     [D]        6       1
2     [A]        5       1
3     [C]        5       1
4  [D, C]        4       2
5  [D, E]        5       2
6  [E, C]        4       2
7  [D, A]        4       2


## Задание 2

*Максимальная оценка задания: **3** балла*

Дана решетка наборов *X<sub>i</sub>* и их частот

![fq](n4_3_1.png)

Минимальный уровень поддержки = **2**

2.1 Используя решетку наборов и их частот, восстановите изначальный набор данных **T**. Отсортируйте **T** по ключу ```(len(Ti), Ti)``` в порядке возрастания

*\* **T** состоит из 10 тидов, все тиды уникальные*

2.2 Для каждого набора *X<sub>i</sub>* вычислите множество **M<sub>i</sub>** индексов тидов *t<sub>j</sub>* ∈ **T**, в которых содержится этот набор: *M<sub>i</sub> = {j | X<sub>i</sub> ⊆ t<sub>j</sub>}*. Получите множество **P** *= {(X<sub>i</sub>, M<sub>i</sub>)}*

2.3 *Самостоятельно* реализуйте алгоритм **Charm**. Используя полученные пары **P** и алгоритм **Charm**, вычислите список всех закрытых наборов **C** (*closed itemsets*). Продемонстрируйте работу алгоритма. Отсортируйте **C** по ключу ```(len(Ci), Ci)``` в порядке возрастания

*\* оформите пункты 2.1, 2.2, 2.3  в виде отдельных функций*

2.4 В исходных данных добавьте +1 к частоте набора ***BCD*** и повторно получите список всех закрытых наборов. Значение минимального уровня поддержки оставьте без изменений.  Повторная демонстрация работы алгоритма **Charm** не требуется

*\* после изменения частоты изначальный набор данных **T** изменится*


In [5]:
data = [('A',7),('B',7),('C',8),('D',7),('E',8),
        ('AB',5),('AC',5),('AD',5),('AE',5),('BC',6),('BD',5),('BE',5),('CD',5),('CE',6),('DE',6),
        ('ABC',4),('ABD',3),('ABE',3),('ACD',3),('ACE',3),('ADE',4),('BCD',3),('BCE',4),('BDE',3),('CDE',4),
        ('ABCD',2),('ABCE',2),('ABDE',2),('ACDE',2),('BCDE',2),('ABCDE',1)]
minSup=0.2
print('Data: ',data)
print('Minsup ratio:',minSup)

Data:  [('A', 7), ('B', 7), ('C', 8), ('D', 7), ('E', 8), ('AB', 5), ('AC', 5), ('AD', 5), ('AE', 5), ('BC', 6), ('BD', 5), ('BE', 5), ('CD', 5), ('CE', 6), ('DE', 6), ('ABC', 4), ('ABD', 3), ('ABE', 3), ('ACD', 3), ('ACE', 3), ('ADE', 4), ('BCD', 3), ('BCE', 4), ('BDE', 3), ('CDE', 4), ('ABCD', 2), ('ABCE', 2), ('ABDE', 2), ('ACDE', 2), ('BCDE', 2), ('ABCDE', 1)]
Minsup ratio: 0.2


In [6]:
def recovT(data_S):
    data = dict(data_S)
    T=[]
    for r_item, r_sup in reversed(data.items()):
        if len(r_item) !=1:
            for item, sup in data.items():
                if set(item).issubset(set(r_item)) and len(item)<len(r_item) and r_sup!=0:
                    data[item]=sup-r_sup    
    for item,sup in data.items():
        if sup>0: T.append(item)
    return T
T = recovT(data)
print('T:',T)

T: ['BD', 'ABC', 'ADE', 'BCE', 'CDE', 'ABCD', 'ABCE', 'ABDE', 'ACDE', 'BCDE', 'ABCDE']


In [7]:
def findIndex(data,T):
    temp_dict = dict(data)
    global_arr=[]
    for key in temp_dict:
        arr_ind=[]
        for index,item in enumerate(T):
            if set(key).issubset(set(item)):
                arr_ind.append(index)
        global_arr.append((key,arr_ind))
    return global_arr
P = findIndex(data,T)
print('P:',P)

P: [('A', [1, 2, 5, 6, 7, 8, 10]), ('B', [0, 1, 3, 5, 6, 7, 9, 10]), ('C', [1, 3, 4, 5, 6, 8, 9, 10]), ('D', [0, 2, 4, 5, 7, 8, 9, 10]), ('E', [2, 3, 4, 6, 7, 8, 9, 10]), ('AB', [1, 5, 6, 7, 10]), ('AC', [1, 5, 6, 8, 10]), ('AD', [2, 5, 7, 8, 10]), ('AE', [2, 6, 7, 8, 10]), ('BC', [1, 3, 5, 6, 9, 10]), ('BD', [0, 5, 7, 9, 10]), ('BE', [3, 6, 7, 9, 10]), ('CD', [4, 5, 8, 9, 10]), ('CE', [3, 4, 6, 8, 9, 10]), ('DE', [2, 4, 7, 8, 9, 10]), ('ABC', [1, 5, 6, 10]), ('ABD', [5, 7, 10]), ('ABE', [6, 7, 10]), ('ACD', [5, 8, 10]), ('ACE', [6, 8, 10]), ('ADE', [2, 7, 8, 10]), ('BCD', [5, 9, 10]), ('BCE', [3, 6, 9, 10]), ('BDE', [7, 9, 10]), ('CDE', [4, 8, 9, 10]), ('ABCD', [5, 10]), ('ABCE', [6, 10]), ('ABDE', [7, 10]), ('ACDE', [8, 10]), ('BCDE', [9, 10]), ('ABCDE', [10])]


In [8]:
def prepareP(P):
    P = dict(P)
    P_comb={}
    for item in P.copy():
        if len(item)>1:
            P_comb[item] = P[item]
            P.pop(item)
    return P,P_comb

def runCharm(P,P_comb,min_sup, C):
    print('\nStart runCharm')
    for item, value in P.items():
        p = {}
        for item_j, value_j in P.items():
            if list(P.keys()).index(item_j)>list(P.keys()).index(item):
                new_item=item+item_j
                if(P_comb.get(new_item)) is not None:
                    new_sup = len(P_comb[new_item])
                    if new_sup >= min_sup*10:
                        print(f'Current element: ({new_item},{P_comb[new_item]})')
                        if value == value_j:
                            P[new_item]=P.pop(item)
                            if item in p: p[new_item]=p.pop(item)
                            P.pop(item_j)
                        elif set(value).issubset(set(value_j)):
                            P[new_item]=P.pop(item)
                            if item in p: p[new_item]=p.pop(item)
                        else:
                            print(f'Add to Pi: ({new_item},{P_comb[new_item]})')
                            p.setdefault(new_item,P_comb[new_item])
        if len(p) != 0: runCharm(p,P_comb,min_sup,C)
        if len(C) != 0:
            for closed_item in C.copy():
                if set(item).issubset(set(closed_item)) is False:
                    C.setdefault(item,value)
                elif value != C[closed_item]:
                    C.setdefault(item,value)
        else: 
            C.setdefault(item,value)
        print(f'Add to C: ({item},{value})')
    C = sorted(C, key=len)
C={}
P,P_comb=prepareP(P)
runCharm(P,P_comb,0.2,C)
C = sorted(C, key=len)
print('\nC=',C)


Start runCharm
Current element: (AB,[1, 5, 6, 7, 10])
Add to Pi: (AB,[1, 5, 6, 7, 10])
Current element: (AC,[1, 5, 6, 8, 10])
Add to Pi: (AC,[1, 5, 6, 8, 10])
Current element: (AD,[2, 5, 7, 8, 10])
Add to Pi: (AD,[2, 5, 7, 8, 10])
Current element: (AE,[2, 6, 7, 8, 10])
Add to Pi: (AE,[2, 6, 7, 8, 10])

Start runCharm
Add to C: (AB,[1, 5, 6, 7, 10])
Add to C: (AC,[1, 5, 6, 8, 10])
Add to C: (AD,[2, 5, 7, 8, 10])
Add to C: (AE,[2, 6, 7, 8, 10])
Add to C: (A,[1, 2, 5, 6, 7, 8, 10])
Current element: (BC,[1, 3, 5, 6, 9, 10])
Add to Pi: (BC,[1, 3, 5, 6, 9, 10])
Current element: (BD,[0, 5, 7, 9, 10])
Add to Pi: (BD,[0, 5, 7, 9, 10])
Current element: (BE,[3, 6, 7, 9, 10])
Add to Pi: (BE,[3, 6, 7, 9, 10])

Start runCharm
Add to C: (BC,[1, 3, 5, 6, 9, 10])
Add to C: (BD,[0, 5, 7, 9, 10])
Add to C: (BE,[3, 6, 7, 9, 10])
Add to C: (B,[0, 1, 3, 5, 6, 7, 9, 10])
Current element: (CD,[4, 5, 8, 9, 10])
Add to Pi: (CD,[4, 5, 8, 9, 10])
Current element: (CE,[3, 4, 6, 8, 9, 10])
Add to Pi: (CE,[3, 4, 6,

In [9]:
newData=dict(data)
newData['BCD']+=1
print('New data',newData)
T = recovT(newData)
print('T=',T)
P = findIndex(data,T)
print('P=',P)
C={}
P,P_comb=prepareP(P)
runCharm(P,P_comb,0.2,C)
C = sorted(C, key=len)
print('\nC=',C)

New data {'A': 7, 'B': 7, 'C': 8, 'D': 7, 'E': 8, 'AB': 5, 'AC': 5, 'AD': 5, 'AE': 5, 'BC': 6, 'BD': 5, 'BE': 5, 'CD': 5, 'CE': 6, 'DE': 6, 'ABC': 4, 'ABD': 3, 'ABE': 3, 'ACD': 3, 'ACE': 3, 'ADE': 4, 'BCD': 4, 'BCE': 4, 'BDE': 3, 'CDE': 4, 'ABCD': 2, 'ABCE': 2, 'ABDE': 2, 'ACDE': 2, 'BCDE': 2, 'ABCDE': 1}
T= ['C', 'ABC', 'ADE', 'BCD', 'BCE', 'CDE', 'ABCD', 'ABCE', 'ABDE', 'ACDE', 'BCDE', 'ABCDE']
P= [('A', [1, 2, 6, 7, 8, 9, 11]), ('B', [1, 3, 4, 6, 7, 8, 10, 11]), ('C', [0, 1, 3, 4, 5, 6, 7, 9, 10, 11]), ('D', [2, 3, 5, 6, 8, 9, 10, 11]), ('E', [2, 4, 5, 7, 8, 9, 10, 11]), ('AB', [1, 6, 7, 8, 11]), ('AC', [1, 6, 7, 9, 11]), ('AD', [2, 6, 8, 9, 11]), ('AE', [2, 7, 8, 9, 11]), ('BC', [1, 3, 4, 6, 7, 10, 11]), ('BD', [3, 6, 8, 10, 11]), ('BE', [4, 7, 8, 10, 11]), ('CD', [3, 5, 6, 9, 10, 11]), ('CE', [4, 5, 7, 9, 10, 11]), ('DE', [2, 5, 8, 9, 10, 11]), ('ABC', [1, 6, 7, 11]), ('ABD', [6, 8, 11]), ('ABE', [7, 8, 11]), ('ACD', [6, 9, 11]), ('ACE', [7, 9, 11]), ('ADE', [2, 8, 9, 11]), ('BCD'

## Задание 3

*Максимальная оценка задания: **2** балла*

Даны последовательности:

| id              | sequence |
| --------------- | -------- |
| *s<sub>1</sub>* | *TATTTATGGTATG*     |
| *s<sub>2</sub>* | *GTACTCTTAGTA*     |
| *s<sub>3</sub>* | *CCCGAGGTGAGCT*     |
| *s<sub>4</sub>* | *GTGTCACATATA*     |
| *s<sub>5</sub>* | *TCGTCTAAAAT*     |

Минимальный уровень поддержки = **4**

3.1 *Самостоятельно* реализуйте алгоритм **GSP** и найдите все подпоследовательности. Продемонстрируйте, как алгоритм перебирает предложенный набор данных. Оформите этот пункт в виде функции

Если будет найдено больше 15 подпоследовательностей, то отобразите только последние (сортировка по количеству символов по возрастанию) 15 значений

3.2 К исходным данным добавьте последовательность ***TCACCC*** и повторно найдите все подпоследовательности. Значение минимального уровня поддержки оставьте без изменений. Повторная демонстрация работы алгоритма не требуется. 

Если будет найдено больше 15 подпоследовательностей, то отобразите только последние (сортировка по количеству символов по возрастанию) 15 значений

In [10]:
def comb(itemSet,u_element):
    comb_item=[]
    for item in itemSet:
        for element in u_element:
            comb_item.append(item+element)
    return comb_item

def returnItemsWithMinSupport(itemSet, transactionList, minSupport):
    supSet={}
    for item in itemSet:
        for transaction in transactionList:
            n=0
            l_item=0
            l_trans=0
            while l_item<len(item) and l_trans<len(transaction):
                if item[l_item]==transaction[l_trans]:
                    n+=1         
                    l_item+=1
                    l_trans+=1
                else:
                    l_trans+=1
            if n == len(item):
                supSet.setdefault(item,0)
                supSet[item]+=1
    sup_out=supSet.copy()
    for item in sup_out.copy():
        new_item=''.join(item)
        sup_out[new_item]=sup_out.pop(item)
    print('After comp sup: ',sup_out)
    supSet={item:v for item,v in supSet.items() if v/10 >= minSupport}
    print('Add to F:',supSet)
    return supSet

def getItemSetTransactionList(itemset):
    transactionList = []
    for record in itemset:
        itemSet=[]
        for item in record:
            itemSet.append(item)
        transactionList.append(itemSet)           
    item_set = set(item for record in itemset for item in record)
    return item_set, transactionList

def runGSP(data_iter, minSupport):
    itemSet, transactionList = getItemSetTransactionList(data_iter)
    print('Unique elements:',[list(item) for item in itemSet])
    resultSet = {}
    print('\n Step 1')
    currentLSet = returnItemsWithMinSupport(itemSet, transactionList, minSupport)
    resultSet.update(currentLSet)
    currentLSet = list(currentLSet)
    k = 2
    while currentLSet != []: 
        items = list(set(item for transaction in currentLSet for item in transaction))
        currentLSet = comb(currentLSet,items)
        print('\n Step %d'% k)
        currentCSet = returnItemsWithMinSupport(currentLSet, transactionList, minSupport)
        resultSet.update(currentCSet)
        currentLSet = list(currentCSet)
        k = k + 1
    return resultSet
sequence=['TATTTATGGTATG','GTACTCTTAGTA','CCCGAGGTGAGCT','GTGTCACATATA','TCGTCTAAAAT']
print('Data:', sequence)
print('Minsup ratio:', 0.4)
result = runGSP(sequence,0.4)
print('\n GSP result:')
printResults(result, 15)

Data: ['TATTTATGGTATG', 'GTACTCTTAGTA', 'CCCGAGGTGAGCT', 'GTGTCACATATA', 'TCGTCTAAAAT']
Minsup ratio: 0.4
Unique elements: [['C'], ['G'], ['A'], ['T']]

 Step 1
After comp sup:  {'C': 4, 'G': 5, 'A': 5, 'T': 5}
Add to F: {'C': 4, 'G': 5, 'A': 5, 'T': 5}

 Step 2
After comp sup:  {'CA': 4, 'CG': 3, 'CC': 4, 'CT': 4, 'GA': 5, 'GG': 4, 'GC': 4, 'GT': 5, 'AA': 5, 'AG': 3, 'AC': 3, 'AT': 5, 'TA': 5, 'TG': 5, 'TC': 4, 'TT': 5}
Add to F: {'CA': 4, 'CC': 4, 'CT': 4, 'GA': 5, 'GG': 4, 'GC': 4, 'GT': 5, 'AA': 5, 'AT': 5, 'TA': 5, 'TG': 5, 'TC': 4, 'TT': 5}

 Step 3
After comp sup:  {'CAA': 4, 'CAG': 2, 'CAC': 2, 'CAT': 4, 'CCA': 4, 'CCG': 2, 'CCC': 1, 'CCT': 4, 'CTA': 4, 'CTG': 2, 'CTC': 3, 'CTT': 4, 'GAA': 4, 'GAG': 3, 'GAC': 3, 'GAT': 5, 'GGA': 4, 'GGG': 2, 'GGC': 2, 'GGT': 4, 'GCA': 3, 'GCG': 1, 'GCC': 2, 'GCT': 4, 'GTA': 5, 'GTG': 4, 'GTC': 4, 'GTT': 5, 'AAA': 4, 'AAG': 3, 'AAC': 1, 'AAT': 5, 'ATA': 4, 'ATG': 3, 'ATC': 2, 'ATT': 4, 'TAA': 4, 'TAG': 3, 'TAC': 3, 'TAT': 5, 'TGA': 5, 'TGG': 2, 

In [11]:
print('Add TCACCC')
sequence.append('TCACCC')
result = runGSP(sequence,0.4)
print('\nGSP result:')
printResults(result,15)

Add TCACCC
Unique elements: [['C'], ['G'], ['A'], ['T']]

 Step 1
After comp sup:  {'C': 5, 'G': 5, 'A': 6, 'T': 6}
Add to F: {'C': 5, 'G': 5, 'A': 6, 'T': 6}

 Step 2
After comp sup:  {'CA': 5, 'CG': 3, 'CC': 5, 'CT': 4, 'GA': 5, 'GG': 4, 'GC': 4, 'GT': 5, 'AA': 5, 'AG': 3, 'AC': 4, 'AT': 5, 'TA': 6, 'TG': 5, 'TC': 5, 'TT': 5}
Add to F: {'CA': 5, 'CC': 5, 'CT': 4, 'GA': 5, 'GG': 4, 'GC': 4, 'GT': 5, 'AA': 5, 'AC': 4, 'AT': 5, 'TA': 6, 'TG': 5, 'TC': 5, 'TT': 5}

 Step 3
After comp sup:  {'CAA': 4, 'CAG': 2, 'CAC': 3, 'CAT': 4, 'CCA': 4, 'CCG': 2, 'CCC': 2, 'CCT': 4, 'CTA': 4, 'CTG': 2, 'CTC': 3, 'CTT': 4, 'GAA': 4, 'GAG': 3, 'GAC': 3, 'GAT': 5, 'GGA': 4, 'GGG': 2, 'GGC': 2, 'GGT': 4, 'GCA': 3, 'GCG': 1, 'GCC': 2, 'GCT': 4, 'GTA': 5, 'GTG': 4, 'GTC': 4, 'GTT': 5, 'AAA': 4, 'AAG': 3, 'AAC': 1, 'AAT': 5, 'ACA': 2, 'ACG': 1, 'ACC': 2, 'ACT': 3, 'ATA': 4, 'ATG': 3, 'ATC': 2, 'ATT': 4, 'TAA': 4, 'TAG': 3, 'TAC': 4, 'TAT': 5, 'TGA': 5, 'TGG': 2, 'TGC': 3, 'TGT': 5, 'TCA': 4, 'TCG': 2, 'TCC':